# Notebook pruebas chatbot

### Modulos instalados

In [ ]:
pip install chromadb

### Llamando funciones y librerias

In [1]:
import funtions_helper as fh

### Cargando documentos

In [2]:
pages = fh.load_files(path_files='documents/', debug=True)

El objeto retornado por la función es de tipo: <class 'list'>. Y cada subobjeto es de tipo: <class 'langchain_core.documents.base.Document'>
En total se cargaron 23 paginas
A continuación se muestran como ejemplo las primeras 5 paginas:
[Document(page_content='Aborda el estudio de la administración con un enfoque humano.\nUna planta docente con formación doctoral.\nAmplia experiencia en investigación aplicada a las organizaciones.\nHa recibido reconocimientos como el Premio AUIP a la Calidad del Posgrado y el \nDoctorado en Iberoamérica (2005) y la Mención de la Asamblea Departamental \n(2014) y el Concejo Municipal.\nEgresados reconocidos y con altas posiciones directivas\nhttp://administr acion. univa lle.edu.co/maest ria-a dministracion\nTels.: (602) 3212100, Ext. 4341, 4769 ::: magisadm@correounivalle.edu.co\nCalle 4B No. 36-00 / Edificio 124 Oficina 2060, Universidad del Valle, Campus San Fernando¿Dónde se oferta?\nCali y Tuluá\nSNIES: 660 MENDuración\n4 semest resTítulo otorgado,

### Fragmentando

In [3]:
splitters = fh.Splitting(obj=pages, chunk_size=500, debug=True)

Los fragmentos son de tamaño: 500 y la superposicion es igual a: 50.0
El número de fragmentos total es igual a: 84


### Embeddings

In [4]:
from langchain_community.vectorstores import Chroma # Guardar indices con los vectores y embeddings de forma local
from langchain_community.embeddings import OllamaEmbeddings # Embeddings con Llama 3
from langchain_core.output_parsers import StrOutputParser # Transformar las respuestas
from langchain_core.runnables import RunnablePassthrough # Ejecutar
import ollama # Modelo llama 3

In [5]:
embedding = OllamaEmbeddings(model='llama3')
vectorstore = Chroma.from_documents(documents=splitters, embedding=embedding, collection_name='indice_chroma')

In [6]:
def Ollama_llm(question, context):
    formatted_prompt = f'Pregunta: {question}\n\nContexto (responde solo sobre el contenido del contexto entregado): {context}'
    response = ollama.chat(model='llama3',messages=[{'role':'user', 'content':formatted_prompt}],options={'temperature':0})
    return response['message']['content']

In [7]:
retriever = vectorstore.as_retriever()
def combine_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [8]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = combine_docs(retrieved_docs)
    return Ollama_llm(question, formatted_context)

In [9]:
result = rag_chain('¿Cuales son los objetivos de la Maestria en Estadistica?')
print(result)

En el contexto de la Maestría en Estadística, los objetivos pueden ser:

1. Desarrollar habilidades en análisis estadístico y resolución de problemas para informar decisiones en salud pública.
2. Aplicar técnicas estadísticas avanzadas para evaluar programas y proyectos sociales en salud.
3. Entrenar a los estudiantes en la capacidad de diseñar, implementar y evaluar estudios epidemiológicos y de investigación en salud.
4. Fomentar la toma de decisiones informadas mediante el uso de datos y análisis estadístico en la gestión de proyectos y programas en salud.
5. Desarrollar habilidades en la comunicación efectiva de resultados estadísticos a diferentes audiencias, incluyendo líderes políticos, profesionales de la salud y comunidades afectadas.
6. Entrenar a los estudiantes en la capacidad de trabajar en equipo y colaborativamente con otros profesionales para resolver problemas en salud pública.
7. Fomentar la reflexión crítica sobre la importancia de la estadística en la toma de decisi